In [27]:
import pandas as pd
customers = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Data Analysis Project November 2025 Youtube\customers.csv")
products = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Data Analysis Project November 2025 Youtube\products.csv")
orders_2023 = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Data Analysis Project November 2025 Youtube\Orders_2023.csv")
orders_2024 = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Data Analysis Project November 2025 Youtube\Orders_2024.csv")
orders_2025 = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Data Analysis Project November 2025 Youtube\Orders_2025.csv")

orders = pd.concat(
    [orders_2023,orders_2024,orders_2025],
    ignore_index=True)
orders_customers = orders.merge(
    customers, on= "CustomerID", how= "left")
main = orders_customers.merge(
    products, on= "ProductID", how="left")
df = main.copy()

In [28]:
#data cleaning 
#understanding the dta
#df.head()
#df.describe(include = 'all')

,OrderID,CustomerID,ProductID,OrderDate,Quantity,Revenue,COGS,SourceFile,Region,CustomerJoinDate,ProductName,ProductCategory,Price,Base_Cost
0,53a9ac5e-e0df-491b-af5a-171c8e3ea288,345e77bf-4a44-4a4c-9b4f-6f5e21e39e8b,1016,2023-01-11,1,43.20,28.28,Orders_2023.csv,East,2023-08-03,Branded Ceramic Mug (Large),Merchandise,43.20,28.28
1,129d9c61-160a-4253-9e87-4a1626963ef1,345e77bf-4a44-4a4c-9b4f-6f5e21e39e8b,1010,2023-12-08,2,210.46,135.90,Orders_2023.csv,East,2023-08-03,Pour-Over Starter Kit,Grinders & Brewers,105.23,67.95
2,0fe79c57-f7e4-40a2-9970-44fb7cbf8188,345e77bf-4a44-4a4c-9b4f-6f5e21e39e8b,1040,2023-11-06,2,48.26,16.90,Orders_2023.csv,East,2023-08-03,Refillable Coffee Pods,Accessories,24.13,8.45
3,16d96526-e2e7-4484-9b1d-f9a80431bce8,345e77bf-4a44-4a4c-9b4f-6f5e21e39e8b,1015,2023-02-02,4,96.52,33.80,Orders_2023.csv,East,2023-08-03,Refillable Coffee Pods,Accessories,24.13,8.45
4,52dfee96-beed-4ac6-935a-81aaa1ccb578,345e77bf-4a44-4a4c-9b4f-6f5e21e39e8b,1020,2023-06-06,1,27.37,21.90,Orders_2023.csv,East,2023-08-03,Minimalist Keychain,Merchandise,27.37,21.90


In [29]:
#standardizing columns
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ", "_")
df.columns

Index(['orderid', 'customerid', 'productid', 'orderdate', 'quantity',
       'revenue', 'cogs', 'sourcefile', 'region', 'customerjoindate',
       'productname', 'productcategory', 'price', 'base_cost'],
      dtype='object')

In [30]:
df = df.rename(columns={'orderid':'order_id','customerid':'customer_id','productid':'product_id','orderdate':'order_date','customerjoindate':'customer_join_date','productname':'product_name','productcategory':'product_category'})
df.columns

Index(['order_id', 'customer_id', 'product_id', 'order_date', 'quantity',
       'revenue', 'cogs', 'sourcefile', 'region', 'customer_join_date',
       'product_name', 'product_category', 'price', 'base_cost'],
      dtype='object')

In [31]:
#fixing dtypes
df['order_date'] = pd.to_datetime(df['order_date'],errors='coerce')
df['customer_join_date'] = pd.to_datetime(df['customer_join_date'],errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4456 entries, 0 to 4455
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            4456 non-null   object        
 1   customer_id         4432 non-null   object        
 2   product_id          4456 non-null   int64         
 3   order_date          3173 non-null   datetime64[ns]
 4   quantity            4456 non-null   int64         
 5   revenue             4415 non-null   float64       
 6   cogs                4456 non-null   float64       
 7   sourcefile          4456 non-null   object        
 8   region              4432 non-null   object        
 9   customer_join_date  4432 non-null   datetime64[ns]
 10  product_name        4456 non-null   object        
 11  product_category    4456 non-null   object        
 12  price               4456 non-null   float64       
 13  base_cost           4456 non-null   float64     

In [32]:
#missing value check
df.isnull().sum()

order_id                 0
customer_id             24
product_id               0
order_date            1283
quantity                 0
revenue                 41
cogs                     0
sourcefile               0
region                  24
customer_join_date      24
product_name             0
product_category         0
price                    0
base_cost                0
dtype: int64

In [33]:
df.duplicated().sum()

np.int64(0)

In [34]:
df = df.dropna(subset=['order_date'])

In [35]:
df = df.dropna(subset=['customer_id'])

In [36]:
#filling revenue
#revenue = quntity*price
null_rev = df['quantity'] * df['price']
df['revenue'] = df['revenue'].fillna(null_rev)
df.isna().sum()

order_id              0
customer_id           0
product_id            0
order_date            0
quantity              0
revenue               0
cogs                  0
sourcefile            0
region                0
customer_join_date    0
product_name          0
product_category      0
price                 0
base_cost             0
dtype: int64

In [38]:
#check if all revenue is positve
df = df[df['revenue'] < 0]
#check if all quantity is positive
df = df[df['quantity'] < 0]

In [ ]:
#Rebuilding time features
df['order_year'] = df['order_date'].dt.year
df['order_month'] = df['order_date'].dt.month
df['order_month_name'] = df['order_date'].dt.month_name()

In [42]:
# exporting df to postgre for EDA
from sqlalchemy import create_engine
#connecting to postgresql
username = 'postgres'
password = '123456'
host = 'localhost'
port = '5432'
database = 'grid_sales'
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")
#loading df into postgresql
table_name = "main" 
df.to_sql(table_name, engine, if_exists="replace", index= False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'main' in database 'grid_sales'.
